In [1]:
!pip install starlette

In [2]:
# Code in this cell is just for (re)starting the API on a Process, and other compatibility stuff with Jupyter cells.
# Just ignore it!
import uvicorn
from multiprocessing import Process

servers = {}
_api_process = None

def start_api(app=None, port=9992, runNew=True):
    """Stop the API if running; Start the API; Wait until API (port) is available (reachable)"""
    assert port in [9991, 9992, 9993, 9994], f'port has unexpected value {port}'
    def run():
        uvicorn.run(app, port=port, host='0.0.0.0', root_path='')    
        
    _api_process = servers.get(port, None)
    if _api_process:
        _api_process.terminate()
        _api_process.join()
        del servers[port]
    
    if runNew:
        assert (not app is None), 'app is None'
        _api_process = Process(target=run, daemon=True)
        _api_process.start()
        servers[port] = _api_process

In [5]:
from starlette.applications import Starlette
from starlette.responses import PlainTextResponse
from starlette.routing import Route, Mount, WebSocketRoute
from starlette.staticfiles import StaticFiles


def homepage(request):
    return PlainTextResponse('Hello, world!')

def user_me(request):
    username = "John Doe"
    return PlainTextResponse(f'Hello, {username}, {data}')

def user(request):
    username = request.path_params['username']
    return PlainTextResponse(f'Hello, {username}, {data}')

async def websocket_endpoint(websocket):
    await websocket.accept()
    await websocket.send_text('Hello, websocket!')
    await websocket.close()

data = {'initialized': False, 'counter': 0}
def startup():
    data['initialized'] = True
    data['counter'] = data['counter'] + 1

def on_shutdown():
    data['initialized'] = False
    
routes = [
    Route('/', homepage),
    Route('/user/me', user_me),
    Route('/user/{username}', user),
    WebSocketRoute('/ws', websocket_endpoint)
]

app = Starlette(debug=True, routes=routes, on_startup=[startup])

start_api(app=app, port=9992, runNew=True)

INFO:     Started server process [1415]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9992 (Press CTRL+C to quit)


INFO:     172.21.0.1:47350 - "GET /user/me HTTP/1.1" 200 OK


INFO:     ('172.21.0.1', 47354) - "WebSocket /gql/" 403
INFO:     connection failed (403 Forbidden)
INFO:     connection closed


INFO:     172.21.0.1:47362 - "GET /user/me HTTP/1.1" 200 OK
INFO:     172.21.0.1:47362 - "GET /user/me HTTP/1.1" 200 OK
INFO:     172.21.0.1:47362 - "GET /user/me HTTP/1.1" 200 OK
INFO:     172.21.0.1:47362 - "GET /user/me HTTP/1.1" 200 OK
INFO:     172.21.0.1:47362 - "GET /user/me HTTP/1.1" 200 OK
INFO:     172.21.0.1:47362 - "GET /user/me HTTP/1.1" 200 OK
INFO:     172.21.0.1:47362 - "GET /user/me HTTP/1.1" 200 OK
INFO:     172.21.0.1:47362 - "GET /user/me HTTP/1.1" 200 OK
INFO:     172.21.0.1:47362 - "GET /user/me HTTP/1.1" 200 OK
INFO:     172.21.0.1:47362 - "GET /user/me HTTP/1.1" 200 OK
INFO:     172.21.0.1:47362 - "GET /user/me HTTP/1.1" 200 OK
INFO:     172.21.0.1:47362 - "GET /user/me HTTP/1.1" 200 OK
INFO:     172.21.0.1:47362 - "GET /user/me HTTP/1.1" 200 OK
INFO:     172.21.0.1:47362 - "GET /user/me HTTP/1.1" 200 OK
INFO:     172.21.0.1:47362 - "GET /user/me HTTP/1.1" 200 OK
INFO:     172.21.0.1:47362 - "GET /user/me HTTP/1.1" 200 OK
INFO:     172.21.0.1:47362 - "GET /user/

INFO:     ('172.21.0.1', 47422) - "WebSocket /gql/" 403
INFO:     connection failed (403 Forbidden)
INFO:     connection closed
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1415]


In [7]:
start_api(app=app, port=9992, runNew=False)

In [8]:
print(data)

{'initialized': False, 'counter': 0}
